# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import openai
import anthropic
import gradio as gr
import dotenv
import os

In [ ]:
class Chatbot:
  def __init__(self, apiKey, publisher='openai'):
    if publisher not in ['openai', 'claude']:
      raise ValueError(f"publisher must be openai or claude, but got {publisher}")
    self.publisher = publisher
    self.systemPrompt = None
    self.historyPrompt = []
    self.llm = openai.OpenAI(api_key=apiKey) if publisher == 'openai' else anthropic.Anthropic(api_key=apiKey)
    
  def setSystemPrompt(self, systemPrompt:str):
    self.systemPrompt = systemPrompt.strip()
    if len(self.historyPrompt) == 0:
      self.historyPrompt.append({"role": "system", "content": f"{systemPrompt}"})
    else:
      self.historyPrompt[0] = {"role": "system", "content": f"{systemPrompt}"}
      
  def _prompt2obj(self, role:str, prompt:str):
    return {
      "role": role,
      "content": prompt.strip()
    }
    
  def unpackText(self, chunk):
    text = ''
    if self.publisher == 'openai':
      text = chunk.choices[0].delta.content or ''
    elif self.publisher == 'claude':
      if chunk.type == "content_block_delta":
        text = chunk.delta.text or ''
        
    return text
      
  def chat(self, message):
    self.historyPrompt.append(self._prompt2obj("user", message))
    completeReply = ""

    if self.publisher == 'openai':
      stream = self.llm.chat.completions.create(model='gpt-4o-mini',
                                            messages=self.historyPrompt,
                                            stream=True)
    elif self.publisher == 'claude':
      stream = self.llm.messages.create(system=self.historyPrompt[0]["content"],
                                          model="claude-sonnet-4-20250514",
                                          max_tokens=200,
                                          messages=self.historyPrompt[1:],
                                          stream=True)
    
    for chunk in stream:
      completeReply += self.unpackText(chunk)
      yield completeReply
      
  
    self.historyPrompt.append(self._prompt2obj("assistant", completeReply))
    
  def _gradioChatWrapper(self):
    def gradioChatFn(message, history):
      for partial_reply in self.chat(message):
        yield partial_reply
    return gradioChatFn
  
  def getAllPrompt(self):
    return self.historyPrompt
    
  def run(self):
    gradioFn = self._gradioChatWrapper()
    gr.ChatInterface(fn=gradioFn, type="messages").launch()
    
  

In [ ]:
# initial env
dotenv.load_dotenv(".env", override=True)
openaiKey = os.getenv("OPENAI_API_KEY")
claudeKey = os.getenv("ANTHROPIC_API_KEY")
openaiInfo = {
  'apiKey': openaiKey,
  'publisher': 'openai'
}
claudeInfo = {
  'apiKey': claudeKey,
  'publisher': 'claude'
}

SYSTEM_PROMPT = """
You are a technical experts and responds every question I asked with an explanation.
"""

openaiChatbot = Chatbot(**openaiInfo)
openaiChatbot.setSystemPrompt(SYSTEM_PROMPT)
openaiChatbot.run()

# claudeChatbot = Chatbot(**claudeInfo)
# claudeChatbot.setSystemPrompt(SYSTEM_PROMPT)
# claudeChatbot.run()
